# MEIRL

In [96]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [97]:
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [98]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [99]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [101]:
start_point_list = []
trajectory_length_list = []
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    start_point_list.append(expert_trajectory[0])
    trajectory_length = len(expert_trajectory)
    trajectory_length_list.append(trajectory_length)
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [102]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [ ]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [ ]:
def update_position(current_position, eps, reward):
    
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        
        total_reward_list = []
        for i in range(len(action_list)):
            immediate_reward = value_list.values[i][0]
            one_step_position = value_list.index[i]
            one_step_action_list = np.array(adjacency_matrix[adjacency_matrix[str(one_step_position)] == 1].index)
            sum_one_step_value = reward_parameter[reward_parameter.index.isin(one_step_action_list)].values.sum()
            total_reward = immediate_reward + (sum_one_step_value * 0.5)
            total_reward_list.append(total_reward)
        updated_position = action_list[np.argmax(total_reward_list)]

    else:
        updated_position = random.choice(value_list["value"].index.tolist())

    return updated_position

In [ ]:
# how many iterations?
Y = []
iteration = 300
for i in range (iteration):
    epsilon = 1 / (i+1)  
    param = 1 / (i+1)
    learner_feature_count["value"] = 0
    for j in range(len(start_point_list)):
        current_position = start_point_list[int(j)]
        trajectory_length = trajectory_length_list[int(j)]
        # agent will walk for 50 steps
        for k in range(trajectory_length):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*param) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
correlation = correlation_dataframe.corr().values[0][1]

In [ ]:
x = np.arange(100)
y = Y[0:100]
plt.plot(x,y, "blue")
filename = "output.png"
plt.savefig(filename)

In [ ]:
print (correlation)

# Local Integration

In [106]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [107]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [108]:
correlation_local_depth

0.24798958383989517

# Connectivity 

In [105]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.18299348460337678

# Global Depth

In [109]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.24798954654952057

# INC

In [115]:
 def INC(current_position):
    in_one_step = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    in_one_step_count = len(in_one_step)
    in_two_step_count = 0
    for i in in_one_step:
        in_two_step = len(np.array(adjacency_matrix[adjacency_matrix[str(i)] == 1].index))
        in_two_step_count = in_two_step_count + in_two_step
    closeness_centrality = in_one_step_count + (in_two_step_count * 2 )
    return closeness_centrality

In [116]:
INC_list = []
for i in adjacency_matrix.index.values:
    INC_list.append(INC(str(i)))
INC_array = np.array(INC_list)
d = {'INC': INC_array}
INC_dataframe = pd.DataFrame(data=d, index=adjacency_matrix.index.values)

In [117]:
relative_asymmetry = (2*(INC_dataframe- 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth =  correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.24332413086681445